In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
df = pd.read_excel("Stock Indexes Daily Realized Covariance (Neural).xlsx", sheet_name= "Vol", index_col="Date",parse_dates=True)

In [3]:
df.head()

,AU200_AUD,FR40_EUR,JP225_USD,UK100_GBP,NAS100_USD,SPX500_USD,US2000_USD
Date,,,,,,,
2005-01-04,0.006543,0.005870,0.007925,0.004362,0.011848,0.007299,0.010940
2005-01-05,0.006181,0.007454,0.005847,0.005601,0.012816,0.007779,0.013023
2005-01-06,0.005293,0.007196,0.010632,0.004218,0.010183,0.007423,0.011491
2005-01-10,0.004436,0.005337,0.007349,0.003858,0.010540,0.006266,0.012405
2005-01-11,0.007802,0.006504,0.006249,0.004878,0.012103,0.006987,0.011026


In [4]:
train=df.iloc[:int(len(df)*0.7)]

In [5]:
test=df.iloc[int(len(df)*0.7)-22:]

In [6]:
df.iloc[int(len(df)*0.7)]

,2016-02-09
AU200_AUD,0.020795
FR40_EUR,0.027200
JP225_USD,0.029546
UK100_GBP,0.021514
NAS100_USD,0.026694
SPX500_USD,0.020427
US2000_USD,0.024958


In [7]:
train=np.array(train)

In [8]:
train.shape

(2270, 7)

In [9]:
RV_d = []
for i in range(len(train)-1):
  RV_d.append(train[i].astype(float))

RV_w = []
for i in range(len(train)-5):
  RV_w.append(np.mean(train[i:i+5]).astype(float))

RV_m = []
for i in range(len(train)-22):
  RV_m.append(np.mean(train[i:i+22]).astype(float))

RV_d=RV_d[len(RV_d)-len(RV_m):]
RV_w=RV_w[len(RV_w)-len(RV_m):]

In [10]:
RV_D = np.zeros([len(RV_d),train.shape[1]])
for i in range(len(RV_d)):
  RV_D[i,:]=RV_d[i]

RV_W = np.zeros([len(RV_w),train.shape[1]])
for i in range(len(RV_w)):
  RV_W[i,:]=RV_w[i]

RV_M = np.zeros([len(RV_m),train.shape[1]])
for i in range(len(RV_m)):
  RV_M[i,:]=RV_m[i]

In [11]:
RV_actuals=train[22:,:]

In [12]:
RV_actuals.shape

(2248, 7)

In [13]:
test=np.array(test)

In [14]:
test.shape

(996, 7)

In [15]:
  for i in range(len(test)-1):
    RV_d.append(test[i].astype(float))

  RV_w = []
  for i in range(len(test)-5):
    RV_w.append(np.mean(test[i:i+5]).astype(float))

  RV_m = []
  for i in range(len(test)-22):
    RV_m.append(np.mean(test[i:i+22]).astype(float))

  RV_d=RV_d[len(RV_d)-len(RV_m):]
  RV_w=RV_w[len(RV_w)-len(RV_m):]
  RV_D_test = np.zeros([len(RV_d),test.shape[1]])
  for i in range(len(RV_d)):
    RV_D_test[i,:]=RV_d[i]

  RV_W_test = np.zeros([len(RV_w),test.shape[1]])
  for i in range(len(RV_w)):
    RV_W_test[i,:]=RV_w[i]

  RV_M_test = np.zeros([len(RV_m),test.shape[1]])
  for i in range(len(RV_m)):
    RV_M_test[i,:]=RV_m[i]

In [16]:
forecasts=np.zeros([test.shape[0]-22,test.shape[1]])
for j in range(test.shape[1]):
  Data ={'Actual': RV_actuals[:,j],
  'D': RV_D[:,j],
  'W': RV_W[:,j],
  'M': RV_M[:,j]}
  dataframe = pd.DataFrame(data=Data)
  rv_scaled=dataframe
  rv_scaled = sm.add_constant(rv_scaled)
  rv_scaled['Actual']=rv_scaled['Actual'].astype(float)
  rv_scaled['D']=rv_scaled['D'].astype(float)
  rv_scaled['W']=rv_scaled['W'].astype(float)
  rv_scaled['M']=rv_scaled['M'].astype(float)
  X = rv_scaled.drop("Actual", axis = 1)
  y = rv_scaled["Actual"]
  results = sm.OLS(y, X).fit()
  Data ={'D': RV_D_test[:,j],
  'W': RV_W_test[:,j],
  'M': RV_M_test[:,j]}
  dataframe = pd.DataFrame(data=Data)
  rv_scaled=dataframe
  X_test = sm.add_constant(rv_scaled)
  forecasts[:,j]= results.predict(X_test)

In [17]:
df1=df.iloc[int(len(df)*0.7):]

In [18]:
df1.head()

,AU200_AUD,FR40_EUR,JP225_USD,UK100_GBP,NAS100_USD,SPX500_USD,US2000_USD
Date,,,,,,,
2016-02-09,0.020795,0.027200,0.029546,0.021514,0.026694,0.020427,0.024958
2016-02-10,0.018632,0.020855,0.033102,0.018034,0.021286,0.017071,0.022575
2016-02-11,0.017475,0.026971,0.029799,0.021625,0.023446,0.019446,0.022890
2016-02-12,0.014229,0.016854,0.033466,0.014669,0.015498,0.012844,0.016731
2016-02-15,0.011773,0.022429,0.021810,0.016740,0.008433,0.007924,0.008424


In [19]:
for i in range(len(df1.columns)):
  df1[df1.columns[i]]=forecasts[:,i]

<ipython-input-19-8a232fbfd1e1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[df1.columns[i]]=forecasts[:,i]


In [20]:
df1.to_excel("HAR Forecasts.xlsx")

In [21]:
test=df.iloc[int(len(df)*0.7):]
test=np.array(test)
RMSE=np.sqrt(np.sum((test-forecasts)**2)/(test.shape[0]*test.shape[1]))
MAE=(np.sum(abs(test-forecasts))/(test.shape[0]*test.shape[1]))
QLIKE=np.sum(test/forecasts-np.log(test/forecasts)-1)/(test.shape[0]*test.shape[1])
MAPE=np.sum(abs(test-forecasts)/(test))/(test.shape[0]*test.shape[1])
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

RMSE: 0.003223514671627422 | QLIKE: 0.040323752756933895 | MAE: 0.00179035055389025 | MAPE: 0.2329384327797009
